In [ ]:
# !pip install openai

import pandas as pd
import os
import time
from tqdm import tqdm
from rouge import Rouge                                     # 모델 성능을 평가 라이브러리
from openai import OpenAI                                   # openai==1.2.0 > requirements.txt에 추가

In [ ]:
# solar client 생성
UPSTAGE_API_KEY = ""                                        # upstage.ai API KEY

client = OpenAI(api_key=UPSTAGE_API_KEY,
                base_url="https://api.upstage.ai/v1/solar")

In [6]:
# 데이터 경로를 지정해줍니다.
DATA_PATH = "../data/raw"
RESULT_PATH = "../outputs/prediction/"

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
train_df.tail()
print(train_df.shape)

(12457, 4)


In [ ]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(DATA_PATH,'dev.csv'))
val_df.tail()
print(val_df.shape)

(499, 4)

## 1. Solar Chat API 요약 성능 확인하기
- Solar Chat API을 이용하여 train 및 validation dataset에 포함된 dialogue 샘플을 요약해 봅니다.

In [ ]:
# 모델 성능에 대한 평가 지표 정의
rouge = Rouge()
def compute_metrics(pred, gold):
    results = rouge.get_scores(pred, gold, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result

In [ ]:
# Prompt 생성 함수
def build_prompt(dialogue):
    system_prompt = "You are an expert in the field of dialogue summarization. Please summarize the following dialogue."

    user_prompt = f"Dialogue:\n{dialogue}\n\nSummary:\n"
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [ ]:
# Summarization 수행 함수
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
    )

    return summary.choices[0].message.content

### (선택) parameter 변경하기
- Solar Chat API를 사용할 때, parameter를 변경하여, 다양한 결과를 얻을 수 있습니다.
- Parameter에 대한 자세한 설명은 [여기](https://developers.upstage.ai/docs/apis/chat#request-body)를 참고해주세요.

In [11]:
def summarization(dialogue):
    summary = client.chat.completions.create(
        model="solar-1-mini-chat",
        messages=build_prompt(dialogue),
        temperature=0.2,
        top_p=0.3,
    )

    return summary.choices[0].message.content

Train Dataset을 이용하여 요약이 잘 되는지 확인해 봅니다.

In [12]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples=3):
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        print(f"Dialogue:\n{dialogue}\n")
        print(f"Pred Summary: {summary}\n")
        print(f"Gold Summary: {row['summary']}\n")
        print("=="*50)

In [13]:
if __name__ == "__main__":
    test_on_train_data()

Dialogue:
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. 
#Person2#: 알겠습니다. 
#Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? 
#Person2#: 네. 
#Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. 
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요. 
#Person1#: 음, 도움 될만한 수업과 약물들이 있습니다. 가시기 전에 더 정보를 드릴게요. 
#Person2#: 네, 고맙습니다, 의사 선생님.

Pred Summary: Dr. Hawkins greets Mr. Smith and asks about his visit. Mr. Smith mentions he is there for a health check-up, which he hasn't had in five years. Dr. Hawkins emphasizes the importance of annual check-ups for early disease detection. Mr. Smith admits to smoking and expresses difficulty quitting. Dr. Hawkins advises him to stop smoking due to health risks and offers information on classes and medications to help him quit before he l

Validation Dataset을 이용하여 요약을 진행하고, 성능을 평가해 봅니다.

In [14]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def validate(num_samples=-1):
    val_samples = val_df[:num_samples] if num_samples > 0 else val_df
    
    scores = []
    for idx, row in tqdm(val_samples.iterrows(), total=len(val_samples)):
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        avg_score = sum(results.values()) / len(results)
        
        scores.append(avg_score)
        
    val_avg_score = sum(scores) / len(scores)

    print(f"Validation Average Score: {val_avg_score}")

In [15]:
if __name__ == "__main__":
    validate(100) # 100개의 validation sample에 대한 요약을 수행합니다.
    
    # 전체 validation data에 대한 요약을 수행하고 싶은 경우 아래와 같이 실행합니다.
    # validate() 

100%|██████████| 100/100 [01:59<00:00,  1.19s/it]

Validation Average Score: 0.09609235277755226


## 2. Solar Chat API로 요약하기
- Solar Chat API을 이용하여 test dataset에 포함된 dialogue를 요약하고 제출용 파일을 생성합니다.

In [16]:
def inference():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

    summary = []
    start_time = time.time()
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        dialogue = row['dialogue']
        summary.append(summarization(dialogue))
        
        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
    
    output = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary" : summary,
        }
    )
    
    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output.to_csv(os.path.join(RESULT_PATH, "output_solar.csv"), index=False)

    return output

In [17]:
if __name__ == "__main__":
    output = inference()

100%|██████████| 499/499 [09:44<00:00,  1.17s/it]


In [18]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,#Person1#은 Ms. Dawson에게 즉시 메시지 사용 금지 정책을 모든 직원...
1,test_1,Person1과 Person2는 교통체증과 출퇴근 시간에 대해 이야기합니다. Per...
2,test_2,"Masha와 Hero가 두 달간의 별거 끝에 이혼 신청을 했으며, Masha가 양육..."
3,test_3,Person1은 Person2의 생일을 축하하며 선물을 주고 파티에 초대받았습니다....
4,test_4,Person1과 Person2는 올림픽 공원의 중심인 올림픽 스타디움에 있습니다. ...
...,...,...
494,test_495,Jack은 Charlie에게 학교 후에 자신의 집에 와서 새로 산 캐릭터 커스터마이...
495,test_496,Person2는 아내와 함께 레코드 플레이어를 구입하고 다양한 레코드를 사기 시작하...
496,test_497,"Person1은 Alice에게 세탁기 사용법을 물어보고, 비누가 필요하다는 것을 알..."
497,test_498,Steve와 Matthew는 오랜만에 만나 인사를 나누었습니다. Steve는 Mat...


## 3. Prompt Engineering
- Prompt engineering을 통해 요약 성능 향상을 시도합니다.

In [19]:
# Few-shot prompt를 생성하기 위해, train data의 일부를 사용합니다.
few_shot_samples = train_df.sample(1)

sample_dialogue1 = few_shot_samples.iloc[0]['dialogue']
sample_summary1 = few_shot_samples.iloc[0]['summary']

print(f"Sample Dialogue1:\n{sample_dialogue1}\n")
print(f"Sample Summary1: {sample_summary1}\n")

Sample Dialogue1:
#Person1#: 거울 한 번 봐. 어때? 
#Person2#: 스타일이 좀 촌스러운 것 같은데. 내가 원하는 건 아니야. 
#Person1#: 이게 너한테 딱 어울리는 것 같아. 최신 패션 스타일이야. 
#Person2#: 아, 그래. 이게 딱 내가 원하던 거야. 이걸로 할게.

Sample Summary1: #Person2#는 거울의 스타일이 촌스럽다고 생각하지만, 최신 패션 스타일이라며 마음에 들어 합니다.



In [20]:
# Prompt를 생성하는 함수를 수정합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    user_prompt = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue.\n\n"
        "Sample Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Sample Summary:\n"
        f"{sample_summary1}\n\n"
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [21]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

Dialogue:
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. 
#Person2#: 알겠습니다. 
#Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? 
#Person2#: 네. 
#Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. 
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요. 
#Person1#: 음, 도움 될만한 수업과 약물들이 있습니다. 가시기 전에 더 정보를 드릴게요. 
#Person2#: 네, 고맙습니다, 의사 선생님.

Pred Summary: Mr. Smith는 5년 만에 Dr. Hawkins를 찾아 건강검진을 받습니다. Dr. Hawkins는 매년 검진의 중요성을 강조하고, Mr. Smith의 눈과 귀는 정상이나 담배 피우는 습관을 걱정합니다. 담배가 폐암과 심장병의 주된 원인임을 알리며 금연을 권하고, 도움이 될 수업과 약물을 제안합니다.

Gold Summary: Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필요성을 안내받고 흡연 습관 개선을 위한 도움을 제안받았습니다.

Dialogue:
#Person1#: 안녕하세요, Mrs. Parker. 잘 지내셨나요?
#Person2#: 안녕하세요, Dr. Peters. 잘 지내고 있어요. Ricky랑 저희 둘 다 백신 맞으러 왔어요.
#Person1#: 알겠습니다. 백신 기록을 보니 Ricky는 소아마비, 파상풍, 그

In [22]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

100%|██████████| 100/100 [01:36<00:00,  1.04it/s]

Validation Average Score: 0.1716829706703118


다른 방식으로 Few-shot sample을 제공하여 Prompt를 구성해 봅니다.

In [24]:
# Few-shot sample을 다른 방식으로 사용하여 prompt를 생성합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    few_shot_user_prompt_1 = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue. Be sure that the summary is simple but captures the essence of the dialogue.\n\n"
        "Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Summary:\n"
    )
    few_shot_assistant_prompt_1 = sample_summary1
    
    user_prompt = (
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": few_shot_user_prompt_1},
        {"role": "assistant", "content": few_shot_assistant_prompt_1},
        {"role": "user", "content": user_prompt},
    ]

In [25]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

Dialogue:
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. 
#Person2#: 알겠습니다. 
#Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? 
#Person2#: 네. 
#Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. 
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요. 
#Person1#: 음, 도움 될만한 수업과 약물들이 있습니다. 가시기 전에 더 정보를 드릴게요. 
#Person2#: 네, 고맙습니다, 의사 선생님.

Pred Summary: Dr. Hawkins는 Mr. Smith에게 5년 만에 건강검진을 받으러 온 것을 상기시키며, 매년 검진의 중요성을 강조합니다. Mr. Smith는 특별히 아픈 곳이 없으면 검진이 필요 없다고 생각했지만, Dr. Hawkins는 심각한 질병을 피하기 위해 미리 발견하는 것이 중요하다고 설명합니다. 검진 중 Dr. Hawkins는 Mr. Smith의 눈과 귀가 괜찮다고 하고, 폐와 심장에 대한 걱정을 표현하며 담배 피우는 습관을 끊으라고 권유합니다. Mr. Smith는 담배를 끊기 어렵다고 말하며, Dr. Hawkins는 도움 될만한 수업과 약물에 대한 정보를 제공하기로 합니다.

Gold Summary: Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필요성을 안내받고 흡연 습관 개선을 위한 도움을 제안받았습니다.

Dialogue

In [26]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

100%|██████████| 100/100 [01:44<00:00,  1.04s/it]

Validation Average Score: 0.15774706187467197


In [ ]:
# Validation Average Score: 0.09609235277755226
# Validation Average Score: 0.1716829706703118
# Validation Average Score: 0.15774706187467197

### (선택) 변경된 Prompt로 test dataset에 대한 요약을 진행합니다.
- 변경된 prompt를 통해 점수가 개선되었다면, test dataset에 대한 요약을 진행하고 제출합니다.

In [27]:
# 변경된 prompt를 사용하여, test data의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    output = inference()

100%|██████████| 499/499 [08:26<00:00,  1.02s/it]


In [28]:
output

,fname,summary
0,test_0,#Person1#은 #Person2#에게 모든 사내 및 외부 통신을 이메일과 공식 ...
1,test_1,#Person1#와 #Person2#는 교통체증과 환경 문제에 대해 이야기합니다. ...
2,test_2,"Masha와 Hero가 두 달간의 별거 끝에 이혼 신청을 했으며, Masha가 양육..."
3,test_3,#Person1#는 #Person2#의 생일을 축하하며 선물을 주고 파티에 초대받습...
4,test_4,#Person1#와 #Person2#는 올림픽 공원의 중심인 올림픽 스타디움을 방문...
...,...,...
494,test_495,Jack은 Charlie에게 학교 끝나고 집에 와서 새로운 캐릭터 커스터마이징 게임...
495,test_496,#Person2#는 아내와 함께 레코드 플레이어를 구입하고 다양한 레코드를 사면서 ...
496,test_497,"Alice는 세탁실을 처음 방문한 #Person1#에게 세탁기와 건조기를 소개하고,..."
497,test_498,"Steve는 Matthew에게 1년 만에 만나 안부를 묻고, Steve는 현재 살 ..."


In [34]:
test_df = pd.read_csv("../data/raw/test.csv")
display(test_df)

,fname,dialogue
0,test_0,"#Person1#: Ms. Dawson, 받아쓰기 좀 부탁드려야겠어요. \n#Per..."
1,test_1,#Person1#: 드디어 왔네! 뭐가 이렇게 오래 걸렸어?\n#Person2#: ...
2,test_2,"#Person1#: Kate, 여기서 일어난 일을 믿기 힘들 거야.\n#Person..."
3,test_3,"#Person1#: 생일 축하해, 이거 너를 위한 선물이야, Brian.\n#Per..."
4,test_4,#Person1#: 이 올림픽 공원 정말 크다! \n#Person2#: 맞아. 지금...
...,...,...
494,test_495,"#Person1#: 얘, Charlie, 학교 끝나고 우리 집에 와서 나랑 비디오 ..."
495,test_496,#Person1#: 어떻게 시골 음악에 관심을 갖게 되었어요?\n#Person2#:...
496,test_497,"#Person1#: 저기, Alice. 여기는 처음 와봤어요. 어떻게 기계를 사용하..."
497,test_498,#Person1#: Matthew? 안녕! \n#Person2#: Steve! 진짜...


In [35]:
output.to_csv("../outputs/output_v1_solar.csv", index=False)